# <left>COVID-19 Dashboard via Plotly</left>
***

## Abstract
_Abstract overview of the analysis and a dashboard_

Coronavirus (COVID-19) confirmed cases and deaths in the United States

***

## Data Source
 _Where datasets come from?_

Datasets provided by **USAFACTS** - 
_Government data from over 70 sources organized to show how the money flows, the impact, and who "the people" are._

***

# Acquiring and Loading Data
_Presenting the code and methods for acquiring and loading data for upcoming analysis_

In [37]:
import pandas as pd
import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import numpy as np
from heapq import nlargest
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score
import datetime as dt
import dash
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash
# pip install geopandas==0.3.0
# pip install pyshp==1.2.10
# pip install shapely==1.6.3

In [38]:
virus_cases_df=pd.read_csv('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv')
virus_cases_df.head(3)

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,974,1002,1015,1030,1052,1066,1073,1073,1096,1113
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,2835,3028,3101,3142,3223,3265,3320,3380,3438,3504


In [39]:
virus_deaths_df=pd.read_csv('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv')
virus_deaths_df.head(3)

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,21,21,21,21,21,21,21,22,22,22
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,21,21,22,22,23,24,24,24,25,25


***

# Data Transformation
_Manipulating datasets, converting it to other formats or structures and getting ready for analysis & dashboard_

In [40]:
#function to self-transform data
def data_transform (x):
    
    #groupping observations by State
    grouped_data=x.groupby(['State']).sum()
    #modifying & cleaning
    grouped_data.drop(grouped_data.columns[0:2], axis=1, inplace=True)
    grouped_data.reset_index(inplace=True)
    #transposing
    tran_data=grouped_data.transpose()
    #modifying & cleaning
    tran_data.reset_index(inplace=True)
    tran_data.columns=tran_data.iloc[0]
    tran_data.rename({'State':'Date'}, axis=1, inplace=True)
    tran_data.drop(tran_data.index[0], inplace=True)
    
    return tran_data
#call
virus_cases_tran_df=data_transform(virus_cases_df)
virus_deaths_tran_df=data_transform(virus_deaths_df)

In [41]:
virus_cases_tran_df.head(3)

,Date,AK,AL,AR,AZ,CA,CO,CT,DC,DE,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
1,1/22/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1/23/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1/24/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [42]:
virus_deaths_tran_df.head(3)

,Date,AK,AL,AR,AZ,CA,CO,CT,DC,DE,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
1,1/22/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1/23/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1/24/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
#function to self-select top ten states
def top_ten_states(x):
    
    #empty dic, list
    max_case_d={}
    state_name=['Date']
    #getting top sum observations
    for i in x.columns[1:]:
        max_val=x[i].max()
        max_case_d[i]=max_val
    #getting top 10 from dic
    ten_highest=nlargest(10, max_case_d, key=max_case_d.get)
    #populating names of states to list
    for i in ten_highest:
        state_name.append(i)
    #returning new df
    virus_df_new=x[state_name]
    
    return virus_df_new
#call
virus_cases_df_top_ten=top_ten_states(virus_cases_tran_df)
virus_deaths_df_top_ten=top_ten_states(virus_deaths_tran_df)

In [44]:
virus_cases_df_top_ten.head(3)

,Date,CA,FL,TX,NY,GA,IL,AZ,NJ,NC,LA
1,1/22/20,0,0,0,0,0,0,0,0,0,0
2,1/23/20,0,0,0,0,0,0,0,0,0,0
3,1/24/20,0,0,0,0,0,1,0,0,0,0


In [45]:
virus_deaths_df_top_ten.head(3)

,Date,NY,NJ,CA,MA,TX,FL,IL,PA,MI,CT
1,1/22/20,0,0,0,0,0,0,0,0,0,0
2,1/23/20,0,0,0,0,0,0,0,0,0,0
3,1/24/20,0,0,0,0,0,0,0,0,0,0


In [46]:
def total_cases_allstates(x):

    #grouping data
    grouped_data=x.groupby(['State']).sum()
    grouped_data.reset_index(inplace=True)
    #only using needed columns
    state_lastcases_df=grouped_data.iloc[:, [0, -1]]
    state_lastcases_df.rename(columns={state_lastcases_df.columns[1]: 'Cases'}, inplace=True)
    asc_df=state_lastcases_df.sort_values(by='Cases', ascending=False)
    return asc_df
#call
virus_total_cases_all_df=total_cases_allstates(virus_cases_df)
virus_total_deaths_all_df=total_cases_allstates(virus_deaths_df)

In [47]:
virus_total_cases_all_df.head(3)

,State,Cases
4,CA,548516
9,FL,518075
43,TX,482136


In [48]:
virus_total_deaths_all_df.head(3)

,State,Cases
34,NY,32442
31,NJ,15849
4,CA,10186


In [49]:
def pie_transform(x):
    #getting top 5
    top5_states=x.State[0:5].tolist()
    top5_cases=x.Cases[0:5].tolist()
    #calc
    calc_df=x.drop(x.index[0:5])
    rest_total_cases=calc_df.Cases.sum()
    #appending total values
    top5_states.append('Other States')
    top5_cases.append(rest_total_cases)
    #creating dedicated pie df
    data_tuples = list(zip(top5_states, top5_cases))
    pie_df=pd.DataFrame(data_tuples, columns=('States', 'Cases'))
    
    return pie_df
#call
pie_df=pie_transform(virus_total_cases_all_df)

In [50]:
def total_cases(x):
    
    #grouping data
    grouped_data=x.groupby(['State']).sum()
    grouped_data.reset_index(inplace=True)
    #only using needed columns
    state_lastcases_df=grouped_data.iloc[:, [0, -1]]
    state_lastcases_df.rename(columns={state_lastcases_df.columns[1]: 'Cases'}, inplace=True)
    state_lastcases_ten_df=state_lastcases_df.nlargest(10, 'Cases')
    asc_df=state_lastcases_ten_df.sort_values(by='Cases', ascending=True)
    return asc_df

#call
virus_total_cases_df=total_cases(virus_cases_df)
virus_total_deaths_df=total_cases(virus_deaths_df)

In [51]:
virus_total_cases_df.head(3)

,State,Cases
18,LA,128746
27,NC,132809
31,NJ,184061


In [52]:
virus_total_deaths_df.head(3)

,State,Cases
6,CT,4441
22,MI,6524
38,PA,7297


In [53]:
def county_total(x,y):

    #getting max observation
    state_max_val=x.Cases.max()
    #getting state name of that max observation
    state_max_name=x.loc[x['Cases']==state_max_val, 'State'].iloc[0]
    state_max_case_df=y.loc[y['State']==str(state_max_name)]
    #dropping unknonwn counties
    state_max_case_df=state_max_case_df[state_max_case_df.countyFIPS != 0]
    #getting total value of the last knonw day
    state_fips_total_df=state_max_case_df.groupby(['countyFIPS']).sum()
    state_fips_total_df.reset_index(inplace=True)
    state_fips_total_df.drop(state_fips_total_df.iloc[:, 1:-1], inplace=True, axis=1)
    state_fips_total_df.rename(columns={state_fips_total_df.columns[1]: 'Total'}, inplace=True)
    
    return state_fips_total_df
#call
state_cases_fips_total=county_total(virus_total_cases_df, virus_cases_df)
state_deaths_fips_total=county_total(virus_total_deaths_df, virus_deaths_df)

In [54]:
state_cases_fips_total.head(3)

,countyFIPS,Total
0,6000,21
1,6001,12136
2,6003,2


In [55]:
state_deaths_fips_total.head(3)

,countyFIPS,Total
0,1,32
1,36001,116
2,36003,2


In [56]:
def daily_total(x):
    
    #grabbing columns into a list
    col_list=x.columns[1:].tolist()
    #summing observations into a new column
    x['Daily_Total']=x[col_list].sum(axis=1)
    new_df=x.iloc[:, [0, -1]]
    
    return new_df
#call
daily_cases_df=daily_total(virus_cases_tran_df)
daily_deaths_df=daily_total(virus_deaths_tran_df)

In [57]:
daily_cases_df.tail(3)

,Date,Daily_Total
197,8/5/20,4786307.0
198,8/6/20,4842827.0
199,8/7/20,4904940.0


In [58]:
daily_deaths_df.tail(3)

,Date,Daily_Total
197,8/5/20,156757.0
198,8/6/20,158022.0
199,8/7/20,159403.0


In [59]:
daily_cases_avg=daily_cases_df.Daily_Total.tolist()
daily_cases_sma_avg_df=pd.DataFrame(daily_cases_avg)
daily_cases_sma_avg_df['SMA_Cases']=round(daily_cases_sma_avg_df.rolling(window=3).mean(), 0)
daily_cases_sma_avg_df.tail(3)

,0,SMA_Cases
196,4786307.0,4732819.0
197,4842827.0,4787201.0
198,4904940.0,4844691.0


In [60]:
daily_deaths_avg=daily_deaths_df.Daily_Total.tolist()
daily_deaths_sma_avg_df=pd.DataFrame(daily_deaths_avg)
daily_deaths_sma_avg_df['SMA_Cases']=round(daily_deaths_sma_avg_df.rolling(window=3).mean(), 0)
daily_deaths_sma_avg_df.tail(3)

,0,SMA_Cases
196,156757.0,155446.0
197,158022.0,156745.0
198,159403.0,158061.0


In [61]:
def ols_reg(x):
    #cutting low observations
    ols_reg_df=x[x['Daily_Total'] > 100000]
    ols_reg_df['X_val']=ols_reg_df.index
    #getting Y,X vals
    Y=ols_reg_df.iloc[:, 1].values.reshape(-1,1)
    X=ols_reg_df.iloc[:, 2].values.reshape(-1,1)
    #running reg
    linear_reg=LinearRegression()
    linear_reg.fit(X,Y)
    Y_pred=linear_reg.predict(X)
    #predicting
    ols_reg_df['YPred']=Y_pred
    ols_reg_df.round({'YPred':0})
    
    return ols_reg_df
#call
ols_reg_df=ols_reg(daily_cases_df)

In [62]:
ols_reg_df.head(3)

,Date,Daily_Total,X_val,YPred
66,3/27/20,102595.0,66,-138244.562521
67,3/28/20,122988.0,67,-105514.320642
68,3/29/20,141786.0,68,-72784.078764


In [63]:
def ols_reg_d(x):
    #cutting low observations
    ols_reg_df=x[x['Daily_Total'] > 1000]
    ols_reg_df['X_val']=ols_reg_df.index
    #getting Y,X vals
    Y=ols_reg_df.iloc[:, 1].values.reshape(-1,1)
    X=ols_reg_df.iloc[:, 2].values.reshape(-1,1)
    #running reg
    linear_reg=LinearRegression()
    linear_reg.fit(X,Y)
    Y_pred=linear_reg.predict(X)
    #predicting
    ols_reg_df['YPred']=Y_pred
    ols_reg_df.round({'YPred':0})
    
    return ols_reg_df
#call
ols_reg_deaths_df=ols_reg_d(daily_deaths_df)

In [64]:
ols_reg_deaths_df.head(3)

,Date,Daily_Total,X_val,YPred
64,3/25/20,1082.0,64,14335.940962
65,3/26/20,1385.0,65,15484.066891
66,3/27/20,1783.0,66,16632.192820


In [65]:
#cheking r2 score 

#y_true=ols_reg_df.Daily_Total.tolist()
#y_pred=ols_reg_df.YPred.tolist()
#r2=round(r2_score(y_true, y_pred, multioutput='variance_weighted'), 3)
#print(r2)

In [66]:
def calc_predictions(x):

    #running ols equation
    fig = px.scatter(x, x="X_val", y="Daily_Total", trendline="ols")
    model = px.get_trendline_results(fig)
    #getting ols params
    params = model.px_fit_results.iloc[0].params
    #getting last real value from dataframe
    last_xval=x.X_val.iloc[-1]
    #calculating the value in 2 weeks
    pred_xval=(last_xval+1)+14
    #getting 'a' value 
    olsreg_aval=params[0]
    #getting 'x' value
    olsreg_xval=params[1]
    #list with range of values
    num_pred_list=list(range(last_xval+1, pred_xval))
    #clear list for prediction values
    pred_val_list=[]
    #calculating predictions
    for i in num_pred_list:
        calc=(olsreg_xval*i)+olsreg_aval
        pred_val_list.append(round(calc, 0))
    #grabbing base date
    base=x.Date.iloc[-1]
    #list of date into 14 days
    date_list=pd.date_range(start=str(base), periods=15)
    date_list=date_list[1:].tolist()
     #stripping time
    date_ymd_list=[]
    for time in date_list:
        date_ymd_list.append(time.strftime('%Y-%m-%d'))
    #creating df with results
    data_tuples = list(zip(date_ymd_list,pred_val_list))
    pred_df=pd.DataFrame(data_tuples, columns=('Dates', 'OLS Values'))
    
    return pred_df
#call
ols_predictions_df=calc_predictions(ols_reg_df)
ols_predictions_deaths_df=calc_predictions(ols_reg_deaths_df)

***

# Exploring and Visualizing Data / Prep Figures for Dashboard
_Exploring the data by analyzing and visualizing the values of features and correlations presented_

In [67]:
#first graph func
def first_graph():
    #figure
    cases_deaths_fig=make_subplots(rows=2, cols=1, vertical_spacing=0.08)

    #setting colors
    state_case_colors=['rgb(0, 255, 204)', 'rgb(0, 230, 184)', 'rgb(0, 204, 163)',
                  'rgb(0, 179, 143)', 'rgb(0, 153, 122)', 'rgb(0, 128, 102)',
              'rgb(0, 102, 82)', 'rgb(0, 77, 61)', 'rgb(0, 51, 41)',
              'rgb(0, 26, 20)']
    state_death_colors=['rgb(255, 0, 0)', 'rgb(230, 0, 0)', 'rgb(204, 0, 0)',
                    'rgb(179, 0, 0)', 'rgb(153, 0, 0)', 'rgb(128, 0, 0)',
                    'rgb(102, 0, 0)', 'rgb(77, 0, 0)', 'rgb(51, 0, 0)',
                    'rgb(26, 0, 0)']

    #assigning colors for cases
    state_cases_name=[]
    for i in virus_cases_df_top_ten.columns[1:]:
        state_cases_name.append(i) 
    state_cases_color_dic=dict(zip(state_cases_name, state_case_colors))
    #assigning colors for deaths
    state_deaths_name=[]
    for i in virus_deaths_df_top_ten.columns[1:]:
        state_deaths_name.append(i)
    state_deaths_color_dic=dict(zip(state_deaths_name, state_death_colors))
    
    #adding case traces to the plot 1
    for i in virus_cases_df_top_ten.columns[1:]:
        cases_deaths_fig.add_trace(go.Scatter(x=virus_cases_df_top_ten.Date, y=virus_cases_df_top_ten[i], name=i, mode='lines', 
                                 line=dict(color=state_cases_color_dic.get(i), width=3), fill='tozeroy',
                                 hovertemplate='<b>Date:</b> %{x} <br><b>Cases:</b> %{y}'),
                                 row=1, col=1
                             )
    #adding death tracess to the plot 2
    for i in virus_deaths_df_top_ten.columns[1:]:
        cases_deaths_fig.add_trace(go.Scatter(x=virus_deaths_df_top_ten.Date, y=virus_deaths_df_top_ten[i], name=i, mode='lines', 
                                 line=dict(color=state_deaths_color_dic.get(i), width=3), fill='tozeroy',
                                 hovertemplate='<b>Date:</b> %{x} <br><b>Deaths:</b> %{y}'),
                                 row=2, col=1
                                 )

    #customizing figure    
    cases_deaths_fig.update_layout(
        title_text=' COVID-19 Cases & Deaths Area Growth in TOP-10 States',
        title_font_family='Arial',
        title_font_color='rgb(74, 160, 207)',
        title={
            'x':0.5,
            'y':0.98
        },
        margin=dict(
            l=25, 
            r=25, 
            t=40, 
            b=25
        ),
        legend=dict(
            #yanchor="top",
            #y=0.98,
            #xanchor="left",
            #x=0.01,
            bgcolor='rgb(0, 51, 77)',
            bordercolor='rgb(0, 68, 102)',
            borderwidth=2,
            font=dict(
                family="Arial",
                size=10,
                color='rgb(74, 160, 207)'
            )
        ),
        showlegend=True,
        plot_bgcolor='rgb(0, 51, 77)',
        paper_bgcolor='rgb(11, 30, 40)'
    )
    cases_deaths_fig.update_xaxes(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(74, 160, 207)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
                family='Arial',
                size=10,
                color='rgb(74, 160, 207)'
        ),
        dtick=16,
        row=1, 
        col=1
    )
    cases_deaths_fig.update_xaxes(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(74, 160, 207)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
                family='Arial',
                size=10,
                color='rgb(74, 160, 207)'
        ),
        dtick=16,
        row=2, 
        col=1
    )
    cases_deaths_fig.update_yaxes(
        title_text='COVID-19 Cases Count',
        title_standoff=1,
        title_font_color='rgb(74, 160, 207)',
        title_font_size=12,
        showline=True,
        showgrid=True,
        gridwidth=0.1,
        gridcolor='rgb(0, 68, 102)',
        linecolor='rgb(74, 160, 207)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(74, 160, 207)'
        ),
        zeroline=False,
        row=1, 
        col=1
    )
    cases_deaths_fig.update_yaxes(
        title_text='COVID-19 Deaths Count',
        title_standoff=8,
        title_font_color='rgb(74, 160, 207)',
        title_font_size=12,
        showline=True,
        showgrid=True,
        gridwidth=0.1,
        gridcolor='rgb(0, 68, 102)',
        linecolor='rgb(74, 160, 207)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(74, 160, 207)'
        ),
        zeroline=False,
        row=2, 
        col=1
    )

    return cases_deaths_fig

#second graph func
def second_graph():

    fig_sma=go.Figure()

    fig_sma.add_trace(go.Indicator(
        mode = "number+delta",
        value = daily_cases_sma_avg_df[0].iloc[-1],
        title = {"text": "<span style='font-size:20;color:rgb(0, 255, 204)'>Total Cases</span>"},
        number = dict(
            prefix="# ",
            font=dict(
                size=30
            )
        ),
        delta = dict(
            position="top",
            reference=daily_cases_sma_avg_df.SMA_Cases.iloc[-1],
            relative=False,
            font=dict(
                size=20
            )
        ),        
        domain = {'x': [0, 1], 'y': [0.5, 1]})
    )

    fig_sma.add_trace(go.Indicator(

        mode = "number+delta",
        value = daily_deaths_sma_avg_df[0].iloc[-1],
        title = {"text": "<span style='font-size:20;color:rgb(255, 0, 0)'>Total Deaths</span>"},
        number = dict(
            prefix="† ",
            font=dict(
                size=30
            )
        ),
        delta = dict(
            position="top",
            reference=daily_deaths_sma_avg_df.SMA_Cases.iloc[-1],
            relative=False,
            font=dict(
                size=20
            )
        ),        
        domain = {'x': [0, 1], 'y': [0, 0.5]})
    )

    fig_sma.update_layout(
        title_text='COVID-19 3-Num SMA',
        title_font_family='Arial',
        title_font_color='rgb(74, 160, 207)',
        title_font_size=9,
        title={
            'x':0.02,
            'y':0.98,
        },
        showlegend=False,
        margin=dict(
            l=0, 
            r=0, 
            t=30, 
            b=0
        ),
        paper_bgcolor='rgb(11, 30, 40)',
        font=dict(
                family="Arial",
                color='rgb(74, 160, 207)'
        )
    )
    
    return fig_sma

#third graph func
def third_graph():
    fig_ols=go.Figure()

    fig_ols.add_trace(go.Scatter(x=ols_reg_df.Date, y=ols_reg_df.Daily_Total,
                             mode='lines', line=dict(color='rgb(0, 255, 204)'),
                             hovertemplate='<b>Date:</b> %{x} <br><b>Cases:</b> %{y}',
                             name='Cases', showlegend=True
                            ))
    fig_ols.add_trace(go.Scatter(x=ols_reg_df.Date, y=ols_reg_df.YPred,
                             mode='lines', line=dict(color='rgb(255, 204, 51)'),
                             hovertemplate='<b>Date:</b> %{x} <br><b>Cases:</b> %{y}',
                             name='OLS', showlegend=True
                            ))

    fig_ols.update_layout(
        title_text='OLS Regression Case Prediction',
        title_font_family='Arial',
        title_font_color='rgb(74, 160, 207)',
        title={
            'x':0.5,
            'y':0.98
        },
        margin=dict(
            l=10, 
            r=10, 
            t=25, 
            b=25
        ),
        legend=dict(
            yanchor="top",
            y=0.90,
            xanchor="left",
            x=0.01,
            bgcolor='rgb(0, 51, 77)',
            bordercolor='rgb(0, 68, 102)',
            borderwidth=2,
            font=dict(
                family="Arial",
                size=10,
                color='rgb(74, 160, 207)'
            )
        ),
        #annotations=[
            #dict(
               # text='<b>R2:</b> '+ str(r2),
               # font=dict(
                #    family='Arial',
               #     size=14,
               #     color='rgb(255, 204, 51)'
               # ),
               # showarrow=False,
               # yanchor="top",
               # yshift=20,
               # xanchor="left",
               # xshift=200
            #)
        #],
        showlegend=True,
        plot_bgcolor='rgb(0, 51, 77)',
        paper_bgcolor='rgb(11, 30, 40)'
    )
    fig_ols.update_xaxes(
        showline=True,
        showgrid=False,
        linecolor='rgb(74, 160, 207)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
                family='Arial',
                size=10,
                color='rgb(74, 160, 207)'
        ),
        dtick=12
    )

    fig_ols.update_yaxes(
        title_text='COVID-19 Cases Count',
        title_standoff=5,
        title_font_color='rgb(74, 160, 207)',
        title_font_size=11,
        showline=True,
        showgrid=True,
        gridwidth=0.1,
        gridcolor='rgb(0, 68, 102)',
        linecolor='rgb(74, 160, 207)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(74, 160, 207)'
        ),
        zeroline=False
    )
    
    return fig_ols

#forth func graph
def fourth_graph():
    fig_table=go.Figure(
        data=[go.Table(
            header=dict(values=list(ols_predictions_df.columns),
                        line_color='rgb(0, 51, 77)',
                        fill_color='rgb(11, 30, 40)',
                        align='left',
                        font=dict(color='rgb(74, 160, 207)', size=12)
                       ),
            cells=dict(values=[ols_predictions_df.Dates, ols_predictions_df['OLS Values']],
                       line_color='rgb(11, 30, 40)',
                       fill_color='rgb(11, 30, 40)',
                       align='left',
                       font=dict(color=['rgb(74, 160, 207)', 'rgb(255, 204, 51)'])
                      )
        )]
    )

    fig_table.update_layout(
        margin=dict(
            l=0, 
            r=0, 
            t=0, 
            b=0
        ),
        paper_bgcolor='rgb(11, 30, 40)'
    )
    
    return fig_table

#fifth func graph
def fifth_graph():
    fig_bar=go.Figure()
    
    state_case_colors=['rgb(0, 255, 204)', 'rgb(0, 230, 184)', 'rgb(0, 204, 163)',
                  'rgb(0, 179, 143)', 'rgb(0, 153, 122)', 'rgb(0, 128, 102)',
              'rgb(0, 102, 82)', 'rgb(0, 77, 61)', 'rgb(0, 51, 41)',
              'rgb(0, 26, 20)']
    state_case_colors.reverse()
    state_death_colors=['rgb(255, 0, 0)', 'rgb(230, 0, 0)', 'rgb(204, 0, 0)',
                    'rgb(179, 0, 0)', 'rgb(153, 0, 0)', 'rgb(128, 0, 0)',
                    'rgb(102, 0, 0)', 'rgb(77, 0, 0)', 'rgb(51, 0, 0)',
                    'rgb(26, 0, 0)']
    state_death_colors.reverse()
    fig_bar.add_trace(go.Bar(x=virus_total_cases_df.State, y=virus_total_cases_df.Cases,
                         marker_color=state_case_colors, marker_line_width=0,
                         marker_line_color='rgb(0, 68, 102)',  
                         hovertemplate='<b>State:</b> %{x} <br><b>Cases:</b> %{y}', name=''
                         )
                 )
    fig_bar.add_trace(go.Bar(x=virus_total_deaths_df.State, y=virus_total_deaths_df.Cases,
                         marker_color=state_death_colors, marker_line_width=0,
                         marker_line_color='rgb(0, 68, 102)',  
                         hovertemplate='<b>State:</b> %{x} <br><b>Cases:</b> %{y}', name='',
                         visible=False
                         )
                 )

    fig_bar.update_layout(
        title_font_family='Arial',
        title_font_color='rgb(74, 160, 207)',
        title={
            'x':0.5,
            'y':0.92
        },
        showlegend=False,
        margin=dict(
            l=10, 
            r=10, 
            t=40, 
            b=25
        ),
        plot_bgcolor='rgb(0, 51, 77)',
        paper_bgcolor='rgb(11, 30, 40)',
        updatemenus = list([
        dict(
             buttons=list([   
                dict(label = 'Cases',
                     method = 'update',
                     args = [{'visible': [True, False]},
                             {'title': 'COVID-19 Cases in TOP-10 States'}]),
                dict(label = 'Deaths',
                     method = 'update',
                     args = [{'visible': [False, True]},
                             {'title': 'COVID-19 Deaths in TOP-10 States'}])
            ]),
            showactive=True,
            y=1.25,
            x=0.1,
            font=dict(
                family="Arial",
                size=14,
                color='rgb(74, 160, 207)'
            )
        )
        ])
    )

    fig_bar.update_xaxes(
        showline=True,
        showgrid=False,
        linecolor='rgb(74, 160, 207)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
                family='Arial',
                size=10,
                color='rgb(74, 160, 207)'
        )
    )

    fig_bar.update_yaxes(
        title_text='COVID-19 # Count',
        title_standoff=1,
        title_font_color='rgb(74, 160, 207)',
        title_font_size=12,
        showline=True,
        showgrid=True,
        gridwidth=0.1,
        gridcolor='rgb(0, 68, 102)',
        linecolor='rgb(74, 160, 207)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(74, 160, 207)'
        ),
        zeroline=False,
    )  
    
    return fig_bar

#sixth func graph
# def sixth_graph():
#     values = state_cases_fips_total.Total.tolist()
#     fips = state_cases_fips_total.countyFIPS.tolist()

#     #getting max observation
#     state_max_val=virus_total_cases_df.Cases.max()
#     #getting state name of that max observation
#     state_max_name=virus_total_cases_df.loc[virus_total_cases_df['Cases']==state_max_val, 'State'].iloc[0]

#     state_case_colors=['rgb(0, 255, 204)', 'rgb(0, 230, 184)', 'rgb(0, 204, 163)',
#                   'rgb(0, 179, 143)', 'rgb(0, 153, 122)', 'rgb(0, 128, 102)',
#                   'rgb(0, 102, 82)', 'rgb(0, 77, 61)', 'rgb(0, 51, 41)',
#                   'rgb(0, 26, 20)']
#     state_case_colors.reverse()
#     endpts=[100,500,1000,5000,10000,
#             50000,100000,250000,500000]

#     plot = ff.create_choropleth(
#         fips=fips, values=values, scope=[state_max_name], show_state_data=True,
#         round_legend_values=True, colorscale=state_case_colors, binning_endpoints=endpts,
#         county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
#         state_outline={'color': 'rgb(255,255,255)', 'width': 0.5}
#     )
    
#     fig_most_cases_map=go.Figure(data=plot)

#     fig_most_cases_map.update_layout(
#         title_text='Most Spread of COVID-19 Cases',
#         title_font_family='Arial',
#         title_font_color='rgb(74, 160, 207)',
#         title={
#             'x':0.5,
#             #'y':0.1
#         },
#         margin=dict(
#             l=5, 
#             r=5, 
#             t=30, 
#             b=5
#         ),
#         legend=dict(
#             #yanchor="top",
#             #y=0.96,
#             #xanchor="left",
#             #x=0.01,
#             bordercolor='rgb(0, 68, 102)',
#             borderwidth=2,
#             font=dict(
#                 family="Arial",
#                 size=8,
#                 color='rgb(74, 160, 207)'
#             )
#         ),
#         paper_bgcolor='rgb(11, 30, 40)',
#         plot_bgcolor='rgb(11, 30, 40)',
#         #have to set height and width dimensions here that overide dimensions in dash?
#         height=280,
#         width=370
#     )
#     fig_most_cases_map.layout.template=None
    
#     return fig_most_cases_map

# #seventh func graph 
# def seventh_graph():
#     values = state_deaths_fips_total.Total.tolist()
#     fips = state_deaths_fips_total.countyFIPS.tolist()

#     #getting max observation
#     state_max_val=virus_total_deaths_df.Cases.max()
#     #getting state name of that max observation
#     state_max_named=virus_total_deaths_df.loc[virus_total_deaths_df['Cases']==state_max_val, 'State'].iloc[0]

#     state_death_colors=['rgb(255, 0, 0)', 'rgb(230, 0, 0)', 'rgb(204, 0, 0)',
#                         'rgb(179, 0, 0)', 'rgb(153, 0, 0)', 'rgb(128, 0, 0)',
#                         'rgb(102, 0, 0)', 'rgb(77, 0, 0)', 'rgb(51, 0, 0)',
#                         'rgb(26, 0, 0)']
#     state_death_colors.reverse()
#     endpts=[100,500,1000,5000,10000,
#             20000,30000,50000,100000]

#     plot = ff.create_choropleth(
#         fips=fips, values=values, scope=[state_max_named], show_state_data=True,
#         round_legend_values=True, colorscale=state_death_colors, binning_endpoints=endpts,
#         county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
#         state_outline={'color': 'rgb(255,255,255)', 'width': 0.5}
#     )
    
#     fig_most_deaths_map=go.Figure(data=plot)

#     fig_most_deaths_map.update_layout(
#         title_text='Most Spread of COVID-19 Deaths',
#         title_font_family='Arial',
#         title_font_color='rgb(74, 160, 207)',
#         title={
#             'x':0.5,
#             #'y':0.1,
#         },
#         margin=dict(
#             l=5, 
#             r=5, 
#             t=30, 
#             b=5
#         ),
#         legend=dict(
#             bordercolor='rgb(0, 68, 102)',
#             borderwidth=2,
#             font=dict(
#                 family="Arial",
#                 size=8,
#                 color='rgb(74, 160, 207)'
#             )
#         ),
#         paper_bgcolor='rgb(11, 30, 40)',
#         plot_bgcolor='rgb(11, 30, 40)',
#         height=280,
#         width=370
#     )
#     fig_most_deaths_map.layout.template=None
    
#     return fig_most_deaths_map

***

# Running Dashboard
_Runnig local dashboard via Plotly Dash_

In [68]:
def inline_dashboard():
    app=JupyterDash(__name__)

    app.layout=html.Div([
        #first main div head
        html.Div([
            html.H2('COVID-19 in USA Dashboard'),
            html.P('COVID-19 is caused by a coronavirus called SARS-CoV-2.\
            Older adults and people who have severe underlying medical conditions\
            like heart or lung disease or diabetes seem to be at higher risk for developing\
            more serious complications from COVID-19 illness.\
            This dashboard retrieves data from USAFacts daily and shows COVID-19 cases & deaths in U.S.')
        ],style={
                'background-color':'rgb(11, 30, 40)',
                'width':'auto',
                'height':'122px',
                'color':'rgb(74, 160, 207)',
                'font-family':'Arial',
                'margin-left':'8px',
                'margin-right':'8px',
                'padding-left':'10px'
                }
        ),
        #second main div
        html.Div([
                dcc.Graph(style={
                'width':'auto',
                'height':'400px'
                },           
                figure=first_graph())
       ],style={
                'margin-left':'8px',
                'margin-right':'8px',
                'margin-top':'12px'

           }
       ),
        #third div left
        html.Div([
                    dcc.Graph(style={
                    'width':'auto',
                    'height':'250px'

                    },           
                    figure=fifth_graph())
                    ],style={
                           'background-color':'rgb(0, 17, 26)',
                            'width':'64%',
                            'margin-top':'12px',
                            'margin-left':'8px',
                            'display':'inline-block'
                            }
                ),
        #third div right
        html.Div([
                    dcc.Graph(style={
                    'width':'auto',
                    'height':'250px'
                    },           
                    figure=second_graph())
                    ],style={
                           'background-color':'rgb(0, 17, 26)',
                            'width':'33%',
                            'margin-top':'12px',
                            'margin-left':'8px',
                            'display':'inline-block'
                            }
                ),
        #forth div left
        html.Div([
                    dcc.Graph(style={
                    'width':'auto',
                    'height':'250px'
                    },           
                    figure=third_graph())
                    ],style={
                           'background-color':'rgb(0, 17, 26)',
                            'width':'69%',
                            'margin-top':'8px',
                            'margin-left':'8px',
                            'display':'inline-block'
                            }
                ),
        #forth div right
        html.Div([
                    dcc.Graph(style={
                    'width':'auto',
                    'height':'250px'
                    },           
                    figure=fourth_graph())
                    ],style={
                           'background-color':'rgb(0, 17, 26)',
                            'width':'29%',
                            'margin-top':'8px',
                            'display':'inline-block'
                            }
                ),
#          html.Div([
#              dcc.Graph(figure=sixth_graph())
#          ], style={
#              'background-color':'yellow',
#              'width':'min-content',
#              'height':'min-content',
#              'margin-left':'8px',
#              'margin-top':'8px',
#              'display':'inline-block'
#          }),
#          html.Div([
#              dcc.Graph(figure=seventh_graph())
#          ], style={
#              'background-color':'yellow',
#              'width':'min-content',
#              'height':'min-content',
#              'margin-left':'10px',
#              'margin-top':'8px',
#              'display':'inline-block'
#          })


    ], style={
        'background-color':'rgb(0, 17, 26)',
        'width':'100%',
        'height':'100%'
    }
    )

    #if __name__ == '__main__':
    return app.run_server(debug=False, mode='inline')

inline_dashboard()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Aug/2020 23:06:56] "GET /_alive_e05a6923-d283-4efe-9be9-e1c780afd406 HTTP/1.1" 200 -


127.0.0.1 - - [08/Aug/2020 23:06:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2020 23:06:56] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2020 23:06:56] "GET /_dash-layout HTTP/1.1" 200 -
